# 智能助手（Assistants）

[智能助手](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) 为开发者提供了一种快速、结构化的方式来调整和版本化 LangGraph 智能体，便于持续迭代实验与测试。

## 为图（Graph）提供配置

我们的 `task_maistro` 图已经完成了接入智能助手所需的配置。

该图包含一个 `configuration.py` 文件，用于声明可以被助手覆盖的可配置字段，并在运行时自动加载。

在图的节点逻辑内部，我们可以通过 `user_id`、`todo_category`、`task_maistro_role` 等字段读取不同助手之间的差异。

## 创建智能助手

回到我们在前面章节中持续构建的 `task_maistro` 应用，它的最佳实践用例是什么？

对我而言，优势在于能够为不同场景维护独立的任务清单。

例如，我希望一个助手专注管理个人生活的待办事项，另一个助手负责工作任务。

这一目标可以通过调整 `todo_category` 与 `task_maistro_role` 这两个可配置字段轻松实现。

![智能助手配置界面截图](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)


In [ ]:
# 安装或升级 LangGraph SDK
# 使用 %%capture 魔法命令隐藏安装过程中的常规输出
# --no-stderr 参数确保出现错误时仍会显示在终端
%%capture --no-stderr
%pip install -U langgraph_sdk


这是部署图时自动创建的默认智能助手，可以作为之后定制助手的参照基线。


In [ ]:
# 导入 LangGraph SDK 客户端工厂
from langgraph_sdk import get_client

# 设置本地部署的 URL 地址
# 这里使用通过 CLI 启动的本地服务
url_for_cli_deployment = "http://localhost:8123"

# 创建客户端实例，用于与部署的图交互
client = get_client(url=url_for_cli_deployment)


### 个人助手

首先我们构建一个专门处理个人事务的智能助手。


In [ ]:
# 创建个人助手
# 调用客户端在部署的图上新建一个智能助手实例
personal_assistant = await client.assistants.create(
    # "task_maistro" 是该部署图的名称
    "task_maistro", 
    # 配置参数：待办类别设置为 "personal"（个人）
    config={"configurable": {"todo_category": "personal"}}
)

# 输出助手的关键信息，确认配置是否生效
print(personal_assistant)


让我们进一步完善这个助手，补充 `user_id` 等专属信息，并[创建一个新版本](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant)。这样可以在不影响线上环境的前提下安全迭代配置。


In [ ]:
# 定义个人助手的角色设定
# 这段系统提示词决定了助手的语气与处理逻辑
task_maistro_role = """你是一位友好且条理清晰的个人任务助手，你的主要职责是帮助用户掌握个人任务和承诺的进度。具体要求：

- 协助跟踪并整理个人任务
- 当需要提供“待办事项摘要”时：
  1. 按截止时间（已逾期、今天、本周、未来）分组列出所有当前任务
  2. 标注任何缺少截止日期的任务，并温和地鼓励用户补充
  3. 提醒那些看起来重要却缺少预计耗时的任务
- 如果新增任务没有截止日期，要主动询问用户是否需要补充
- 在帮助用户保持自我管理的同时，始终保持支持性的语气
- 根据截止时间和重要性协助用户排列任务优先级

你的沟通风格应该积极、乐于助人，而不是评判。

当任务缺少截止日期时，可以这样回应：“我注意到 [任务] 还没有截止日期。要不要补充一个，帮助我们更好地追踪？”"""

# 组合需要更新的配置项
configurations = {"todo_category": "personal", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

# 更新个人助手配置
personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"],
    config={"configurable": configurations}
)

# 打印更新后的助手信息
print(personal_assistant)


### 工作助手

接下来再创建一个面向工作任务的助手，帮助我跟进团队安排。


In [ ]:
task_maistro_role = """你是一位专注且高效的工作任务助手。

你的核心职责是帮助用户以现实可行的时间表管理工作承诺。

具体要求：

- 协助跟踪并整理工作任务
- 当提供“待办事项摘要”时：
  1. 按截止时间（已逾期、今天、本周、未来）分组列出所有当前任务
  2. 标注任何缺少截止日期的任务，并温和地提醒用户补充
  3. 提醒那些看起来重要却缺少预计耗时的任务
- 与用户讨论新任务时，结合任务类型给出合理的时间建议：
  - 开发者关系相关功能：通常需要 1 天
  - 课程课件评审或反馈：通常需要 2 天
  - 文档冲刺：通常需要 3 天
- 根据截止日期和团队依赖关系帮助用户安排优先级
- 在协助用户履行承诺时保持专业、务实的语气

当任务缺少截止日期时，可以这样回应：“我注意到 [任务] 还没有截止日期。参考类似的任务，这类工作大约需要 [建议时长]。要不要据此设定一个截止日期？”"""

configurations = {"todo_category": "work", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

# 创建工作助手
# 使用客户端创建一个新的工作助手实例
work_assistant = await client.assistants.create(
    # "task_maistro" 是我们部署的图的名称
    "task_maistro", 
    # 配置参数：待办类别设置为 "work"（工作）
    config={"configurable": configurations}
)

# 打印创建的工作助手信息
print(work_assistant)


## 使用智能助手

智能助手的配置被持久化在部署所使用的 `Postgres` 数据库中。借助 SDK，我们可以方便地[检索](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)并管理这些助手。


In [ ]:
# 搜索所有智能助手
# 使用客户端搜索功能获取所有已创建的助手
assistants = await client.assistants.search()

# 遍历并打印每个助手的基本信息
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],  # 助手的唯一标识符
        'version': assistant['version'],            # 助手的版本号
        'config': assistant['config']               # 助手的配置信息
    })

我们还可以通过 SDK 维护这些助手，例如删除不再使用的测试助手。

> 视频中的示例语法已经过时。下面的代码展示了更新后的做法：先创建一个临时助手，再将其删除。


In [ ]:
# 创建一个临时的智能助手
# 用于演示删除功能
temp_assistant = await client.assistants.create(
    "task_maistro", 
    config={"configurable": configurations}
)

# 搜索所有助手并显示删除前的状态
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"删除前: {{'assistant_id': {assistant['assistant_id']}}}")
    
# 删除我们的临时助手
# 删除最后一个助手（刚创建的临时助手）
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

# 再次搜索所有助手并显示删除后的状态
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"删除后: {{'assistant_id': {assistant['assistant_id']} }}")

接下来，保存个人助手和工作助手的 ID，方便后续调用。


In [ ]:
# 设置助手ID变量，方便后续使用
# 从搜索结果中获取工作助手和个人助手的ID
work_assistant_id = assistants[0]['assistant_id']      # 工作助手的ID
personal_assistant_id = assistants[1]['assistant_id']  # 个人助手的ID

### 工作助手

先让工作助手处理几条任务，观察它的响应。


In [ ]:
# 导入必要的消息类型和转换工具
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

# 用户输入：创建或更新几条待办事项
user_input = "请帮我创建或更新几条待办事项：1）今天下班前重新录制第 6 模块第 5 课。2）在下周一之前更新 audioUX。"

# 创建一个新的对话线程
# 线程用于管理用户与助手之间的完整对话历史
thread = await client.threads.create()

# 使用工作助手处理用户输入
# 通过流式处理持续获取助手的响应
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 当接收到值事件时，处理并显示助手的响应
    if chunk.event == 'values':
        state = chunk.data
        # 将消息转换为可读格式并打印最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


In [ ]:
# 为工作助手新增一条待办
user_input = "再创建一条待办：完成报表生成教程的最终稿。"

# 新建对话线程，用于承载对话上下文
thread = await client.threads.create()

# 流式运行：将用户消息发送给工作助手，并逐步读取返回值
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 当事件类型为 values 时，代表有新的状态返回
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


助手会按照它的系统提示来约束输出，例如在创建任务时提醒我补充截止日期。


In [ ]:
# 回复助手：为该任务设置下周二为截止日期
user_input = "好的，这项任务就安排在下周二完成。"

# 继续在同一个线程中进行流式交互
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理返回的状态值
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


### 个人助手

同样的流程也适用于个人助手，我们可以为它创建或查询待办事项。


In [ ]:
# 为个人助手批量创建两条待办
user_input = "请创建两条待办：1）本周末了解宝宝游泳课的安排。2）为冬季出行确认 AmEx 积分。"

# 新建线程并以流式方式与个人助手交互
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理返回值事件
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


In [ ]:
# 让个人助手给出当前待办摘要
user_input = "请给我一个待办事项摘要。"

# 新建线程并请求摘要
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理流式返回并打印最终消息
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()
